# Testing Queries

Note: For now, we are lucky that the imports work, because we are using relative imports, and at one point, it will cause errors.

## IMPORTS

In [1]:
import os
import sys
import random
import requests
import argparse
from typing import List, Set

import numpy as np
import pandas as pd

In [2]:
from db.yagodb import YagoDB
from db.constants.main import YAGO_ALL_ENTITY_COUNT, YAGO_FACTS_ENTITY_COUNT
from db.functions.entity import get_random_entities_query

In [3]:
from kg.query import get_triples_multiple_subjects_query, get_description_multiple_entities_query, query_kg, get_triples_from_response

In [4]:
from utils.constants import YAGO_ENTITY_STORE_DB_PATH, YAGO_PREFIXES_PATH, YAGO_ENDPOINT_URL
from utils.prefix import get_prefixes, get_url_from_prefix_and_id
from utils.random_walk2 import SPARQL_COLUMNS_DICT, RandomWalk2

### CONSTANTS

### FUNCTIONS

### Experiment Single Walks

In [5]:
yago_db = YagoDB(YAGO_ENTITY_STORE_DB_PATH)

In [6]:
random_walk_2 = RandomWalk2(yago_db)

In [8]:
entities_df = random_walk_2.random_walk_description_batch(num_of_entities=10, depth=3)

In [9]:
entities_df

,entity0,description0,predicate1,entity1,description1,predicate2,entity2,description2
0,http://yago-knowledge.org/resource/Haddington_...,"architectural structure in East Lothian, Scotl...",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/HumanMadeGe...,alterations to land created by people,http://www.w3.org/2002/07/owl#sameAs,http://yago-knowledge.org/resource/HumanMadeGe...,alterations to land created by people
1,http://yago-knowledge.org/resource/Arctictenas...,genus of fishes,http://schema.org/parentTaxon,http://yago-knowledge.org/resource/Ctenaspidid...,family of fishes,http://schema.org/parentTaxon,http://yago-knowledge.org/resource/Cyathaspidida,order of fishes (fossil)
2,http://yago-knowledge.org/resource/Kedarpur_Br...,NaN,http://yago-knowledge.org/resource/ownedBy,http://yago-knowledge.org/resource/India_Post,Government-operated Postal system in India,http://schema.org/location,http://yago-knowledge.org/resource/New_Delhi,capital of India
3,http://yago-knowledge.org/resource/Islamic_Fou...,"school in Villa Park, DuPage County, Illinois,...",http://schema.org/location,http://yago-knowledge.org/resource/United_States,country in North America,http://schema.org/memberOf,http://yago-knowledge.org/resource/Universal_P...,specialized agency of the United Nations focus...
4,http://yago-knowledge.org/resource/Elke_Franck...,NaN,http://schema.org/gender,http://yago-knowledge.org/resource/Female__u00...,to be used in 'sex or gender' (P21) to indicat...,http://www.w3.org/2000/01/rdf-schema#label,žeńska,NaN
5,http://yago-knowledge.org/resource/Sophora_pro...,species of plant,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Taxon,"group of one or more organism(s), which a taxo...",http://www.w3.org/2002/07/owl#disjointWith,http://schema.org/CreativeWork,artistic creation
6,http://yago-knowledge.org/resource/Huaxi_Shuik...,reservoir in People's Republic of China,http://schema.org/location,http://yago-knowledge.org/resource/Fujian,province of China,http://yago-knowledge.org/resource/capital,http://yago-knowledge.org/resource/Fuzhou,"capital city of Fujian province, China"
7,http://yago-knowledge.org/resource/Lazar_Paču,NaN,None,None,NaN,None,None,NaN
8,http://yago-knowledge.org/resource/Afgan-Memor...,NaN,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Website,set of related web pages served from a single ...,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Online_Publ...,work manifested in Internet
9,http://yago-knowledge.org/resource/Ogle_Blg-Ec...,eclipsing binary star,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Eclipsing_b...,binary star in which the orbit plane of the tw...,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Binary_star,star system consisting of two stars
